# Agente de Búsqueda Inteligente
Sistema que combina documentos locales en pdf, json y búsqueda web para respuestas precisas

## 1. Configuración Inicial

- Carga claves API desde `.env`
- Inicializa el modelo de lenguaje GPT-3.5

In [1]:
# Celda 1: Configuración inicial
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict

load_dotenv()

api_key = os.getenv("API_KEY")
api_key_tavily = os.getenv("TAVILY_API_KEY")
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key, temperature=0, verbose=True)

## 2. Motor de Búsqueda en PDFs

- Carga múltiples PDFs
- Usa embeddings para búsqueda semántica
- Provee respuestas con refencias a páginas
- Herramienta de PDF

In [10]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

class BusquedaPDF:
    def __init__(self, pdf_paths: list[str]):
        docs = []
        for path in pdf_paths:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Archivo no encontrado: {path}")
            loader = PyMuPDFLoader(path)
            docs.extend(loader.load())
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        frag = splitter.split_documents(docs)
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        store = FAISS.from_documents(frag, embeddings)
        self.qa = RetrievalQA.from_chain_type(
            llm=ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key),
            chain_type="stuff",
            retriever=store.as_retriever(search_kwargs={"k":3}),
            return_source_documents=True
        )
    def run(self, query: str) -> str:
        res = self.qa.invoke({"query":query})
        answer = res.get("result","")
        docs = res.get("source_documents",[])
        if docs:
            fuentes = "\n".join(f"- {d.metadata.get('source','pdf')}" for d in docs)
            return f"{answer}\n\nFuentes:\n{fuentes}"
        return answer

# Instanciar BusquedaPDF y herramienta
from langchain.tools import Tool
rutas_pdfs=["Investigación de WindSurf.pdf","nke-10k-2023.pdf"]
buscador_pdf=BusquedaPDF(rutas_pdfs)
tool_pdf=Tool(
    name="busqueda_pdf",
    func=buscador_pdf.run,
    description="Busca en documentos PDF",
    metadata={"category":"Documentos locales","file_types":[".pdf"]}
)

FileNotFoundError: Archivo no encontrado: Investigación de WindSurf.pdf

## 3. Motor de Búsqueda en JSONs

- Crea clase de busqueda JSONs
- Crea el prompt como guia para el agente
- Analiza contexto de la pregunta
- Usa ejemplos para mejor precisión

- Crea la herramienta JSON

In [3]:
import json
from langchain.schema import SystemMessage, HumanMessage

class BusquedaJSON:
    def __init__(self, json_paths: list[str]):
        # Cargamos el AST completo
        combined = {}
        for path in json_paths:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Archivo no encontrado: {path}")
            with open(path, encoding="utf-8") as f:
                data = json.load(f)
            combined.update(data)
        self.ast = combined

    def run(self, query: str) -> str:
        # Construimos prompt para el LLM que contiene el AST
        ast_str = json.dumps(self.ast, indent=2, ensure_ascii=False)
        prompt = [
            SystemMessage(content="Eres un asistente experto en análisis de AST de código Python."),
            HumanMessage(content=(
                f"Se te proporciona el siguiente AST en formato JSON de un archivo con definiciones de funciones, "
                f"variables, comentarios y clases:\n{ast_str}\n\n"
                f"Indica primero la lista de funciones que contiene el AST (nombre y parámetros), "
                f"y luego explica brevemente el propósito de cada función.\n"
                f"Consulta del usuario: {query}"
            ))
        ]
        respuesta = model(prompt).content
        return respuesta

# Instanciar BusquedaJSON y herramienta
rutas_json=["ast_summary.json"]
buscador_json=BusquedaJSON(rutas_json)
tool_json=Tool(
    name="busqueda_json",
    func=buscador_json.run,
    description="Analiza un AST JSON y explica sus funciones",
    metadata={"category":"Documentos locales","file_types":[".json"]}
)

## 4. Herramienta de Búsqueda Web

- Acceso a información actualizada
- Resultados de alta relevancia
- Filtrado de contenido irrelevante

In [4]:
from langchain_tavily import TavilySearch
tavily=TavilySearch(max_results=5,topic="general")
def busqueda_internet(q:str)->str:
    res=tavily.invoke({"query":q}).get("results",[])
    return "\n\n".join(r.get("content","") for r in res[:3]) or "Sin resultados"

tool_web=Tool(
    name="busqueda_internet",
    func=busqueda_internet,
    description="Busca en internet",
    metadata={"category":"Búsqueda en vivo","sources":["Tavily"]}
)

## 5. Núcleo de Decisión

- Crea los nodo del Grafo
- Previene errores de routing

In [5]:
from langgraph.graph import StateGraph
from langchain.schema import SystemMessage as SysMsg, HumanMessage as HumMsg

class AgentState(TypedDict): input:str; tool_used:str; output:str; next_step:str

def decide_tool(state:AgentState)->AgentState:
    text=state["input"].lower()
    if any(k in text for k in ["windsurf","pdf"]): return {"next_step":"usar_pdf"}
    if any(k in text for k in ["json","ast","funcion","variable","comentario","clase"]): return {"next_step":"usar_json"}
    return {"next_step":"usar_web"}

def usar_pdf(state:AgentState)->AgentState:
    out=buscador_pdf.run(state["input"])
    return {"input":state["input"],"tool_used":"busqueda_pdf","output":out}

def usar_json(state:AgentState)->AgentState:
    out=buscador_json.run(state["input"])
    return {"input":state["input"],"tool_used":"busqueda_json","output":out}

def usar_web(state:AgentState)->AgentState:
    out=busqueda_internet(state["input"])
    return {"input":state["input"],"tool_used":"busqueda_internet","output":out}

# Montar y compilar grafo
graph=StateGraph(AgentState)
graph.add_node("decision",decide_tool)
graph.add_node("usar_pdf",usar_pdf)
graph.add_node("usar_json",usar_json)
graph.add_node("usar_web",usar_web)
graph.add_node("fin",lambda s:s)

graph.set_entry_point("decision")
graph.add_conditional_edges("decision",lambda s:s["next_step"],{"usar_pdf":"usar_pdf","usar_json":"usar_json","usar_web":"usar_web","fin":"fin"})
for n in ["usar_pdf","usar_json","usar_web"]: graph.add_edge(n,"fin")
agent_executor=graph.compile()


## 5. Interfaz de Chatbot

- Loop conversacional continuo
- Soporta comandos de salida
- Muestra metadata de ejecución

In [6]:
# Celda 5: Chatbot interactivo
print("Chatbot de LangGraph - escriba 'salir' para terminar.")

while True:
    pregunta = input("Usuario: ")
    
    if pregunta.lower() in ["salir", "exit", "quit"]:
        print("Fin de la conversación.")
        break
    
    entrada_agente = {"input": pregunta, "tool_used": "none", "output": ""}
    resultado = agent_executor.invoke(entrada_agente)
    
    print("\nRespuesta:", resultado["output"])
    print("Herramienta usada:", resultado["tool_used"])
    print("-" * 40)
   

Chatbot de LangGraph - escriba 'salir' para terminar.

Respuesta: Sin resultados
Herramienta usada: busqueda_internet
----------------------------------------


C:\Users\6003465\AppData\Local\Temp\ipykernel_22232\1204031144.py:29: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = model(prompt).content



Respuesta: La lista de funciones que contiene el AST es la siguiente:

1. Función "saludar":
   - Nombre: "saludar"
   - Parámetros: "(nombre)"
   - Docstring: null
   - Llamadas a otras funciones: ["print"]

Propósito de cada función:
1. Función "saludar": Esta función recibe un parámetro "nombre" y su propósito es imprimir un saludo utilizando el valor del parámetro "nombre" mediante la función "print".

En cuanto a la función "saludar" del JSON proporcionado, puedo decirte que:
- La función "saludar" tiene como nombre "saludar".
- La función "saludar" recibe un parámetro llamado "nombre".
- La función "saludar" no tiene un docstring asociado.
- La función "saludar" realiza una llamada a la función "print".
Herramienta usada: busqueda_json
----------------------------------------

Respuesta: Thor (en nórdico antiguo: Þórr) es el dios nórdico del trueno, el cielo y la agricultura. Es hijo de Odín, jefe de los dioses, y de la consorte de Odín, Jord (Tierra), y esposo de la diosa de la